In [2]:
!pip install mxnet 

  Using cached https://files.pythonhosted.org/packages/53/39/4ab213673844e0c004bed8a0781a0721a3f6bb23eb8854ee75c236428892/graphviz-0.8.4-py2.py3-none-any.whl
  Found existing installation: numpy 1.15.4
    Uninstalling numpy-1.15.4:
      Successfully uninstalled numpy-1.15.4


DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [ ]:
!pip install opencv-python

In [ ]:
from mxnet import autograd, gluon, image, init
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms
from matplotlib import pyplot as plt
from shutil import copyfile
import os
import numpy as np
import random

In [ ]:
from pathlib import Path
try:
    os.makedirs('train')
except OSError as e:
    pass

train_list = open("./train_masked.txt", "r").read().splitlines()

In [24]:
def color_changer(img, mask):
  # Color Jitters 
    color_brightness = transforms.RandomApply(transforms.RandomBrightness(0.25))
    color_contrast = transforms.RandomApply(transforms.RandomContrast(0.25))
    color_saturation = transforms.RandomApply(transforms.RandomSaturation(0.25))
    color_hue = transforms.RandomApply(transforms.RandomHue(0.25))
    # add noise
    add_noise = transforms.RandomApply(transforms.RandomLighting(alpha=0.5))

    color_aug = transforms.Compose([color_brightness, color_contrast, color_saturation, color_hue, add_noise])

    return color_aug(img), mask

In [21]:
def random_crop_flip(img, mask):
    img = img.asnumpy()

    ori_W = img.shape[1]
    ori_H = img.shape[0]
    new_W = int(ori_W/2)
    new_H = int(ori_H/2)

    crop_rot = random.randint(0,1)

    # randomly apply random crop
    if crop_rot == 1:
        x = random.randint(0,new_W)
        y = random.randint(0,new_H)
        img = img[y:y+new_H, x:x+new_W]
        mask = mask[y:y+new_H, x:x+new_W, :]

    # randomly apply random rotate
    else: 
        img = np.rot90(img)
        mask = np.rot90(mask)
        y = random.randint(0,ori_W-new_H)
        x = random.randint(0,ori_H-new_W)
        img = img[y:y+new_H, x:x+new_W]
        mask = mask[y:y+new_H, x:x+new_W, :]

    # randomly apply random flip
    # if random.randint(0,1) == 1: 
    #    img = np.fliplr(img)
    #    mask = np.fliplr(mask)

    return img, mask

In [26]:
MAX_IDX = 4

for dirname, dirnames, filenames in os.walk('.'):
    if dirname == './data':
        for img_shortname in train_list:
            img_name = './data/' + img_shortname + '.jpg'
            npz_name = './data/' + img_shortname + '.npz'

        for idx in range(MAX_IDX):
            # load image and mask
            img = image.imread(img_name)
            mask = np.load(npz_name)['arr_0']

            img, mask = color_changer(img, mask)
            img, mask = random_crop_flip(img, mask)

            new_name = './train/' + img_shortname + '_' + str(idx) + '.jpg'
            plt.imsave(new_name, img)
            new_npz_name = './train/' + img_shortname + '_' + str(idx)
            np.savez_compressed(new_npz_name, mask)

In [27]:
for dirname, dirnames, filenames in os.walk('.'):
    if dirname == './data':
        for img_shortname in train_list:
            if img_shortname:
            # generate image 
            img_name = './data/' + img_shortname + '.jpg'
            img_copy_name = './train/' + img_shortname + '.jpg'
            copyfile(img_name, img_copy_name)

            npz_name = './data/' + img_shortname + '.npz'
            npz_copy_name = './train/' + img_shortname + '.npz'
            copyfile(npz_name, npz_copy_name)

# Move the testing set to "test" directory




















In [28]:
try:
    os.makedirs('test')
except OSError as e:
    pass

test_masked = open('test_masked.txt', 'r')
test = test_masked.read()
test_list = test.split("\n")


for dirname, dirnames, filenames in os.walk('.'):
    if dirname == './data':
        for i in test_list:
            if i:
            # generate image 
            img_shortname = i
            img_name = './data/' + img_shortname + '.jpg'
            img_copy_name = './test/' + img_shortname + '.jpg'
            copyfile(img_name, img_copy_name)

            npz_name = './data/' + img_shortname + '.npz'
            npz_copy_name = './test/' + img_shortname + '.npz'
            copyfile(npz_name, npz_copy_name)
